# Brazilian COVID-19 vaccination progress
### Data Engineering Capstone Project

#### Project Summary
This project is meant to apply everything I on Udacity's Data Engineering nanodegree, perfoming efficient ETL on large datasets follow a process that when finished will support an high perfomance analytical platform over such datasets.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, monotonically_increasing_id, split, substring, to_date
from pyspark.sql.types import IntegerType

In [2]:
spark = SparkSession.builder.getOrCreate()

### Step 1: Project Scope and Datasets

#### Scope
The goal here is to load COVID-19 vaccination data, made available by the Brazilian government, to assess the progress made so far. In addition to data, we want to segment the progress geographically, from macroregions down to cities. With that, we would be able to inform stakeholders and support decisions regarding vaccine distribution around the country, looking at parameters such as demographic density, which is a relevant factor for the virus spread.

#### Describe and Gather Data 
There are three sources of data:

- **Vaccination**: this dataset is updated daily with data of each vaccine applied in Brazil so far, which means that the data is appended. It's possible to download data from certain states or the whole country at once. Over 20 millions doses have been applied as of April 13th 2021, so we'll use dataset from a few states only as a sample here. Available at: https://opendatasus.saude.gov.br/dataset/covid-19-vacinacao/resource/ef3bd0b8-b605-474b-9ae5-c97390c197a8
- **City Population**: this dataset contains IBGE's brazilian population estimates for 2020, by city. Available at: https://sidra.ibge.gov.br/geratabela?format=ods&name=tabela6579.ods&terr=NC&rank=-&query=t/6579/n1/all/n6/all/v/all/p/last%201/l/v,p,t
- **Demographic density**: the last dataset, also from IBGE, has the demographic density for each city in Brazil. Available at: https://sidra.ibge.gov.br/geratabela?format=ods&name=tabela1301.ods&terr=NC&rank=-&query=t/1301/n6/all/v/616/p/all/d/v616%202/l/v,p,t

In [3]:
demographic_density_df = pd.read_excel('data/IBGE/tabela1301.ods', engine='odf', skiprows=3, skipfooter=1, names=['city_id', 'city_and_state', 'demographic_density'])
demographic_density_df

,city_id,city_and_state,demographic_density
0,1100015,Alta Floresta D'Oeste (RO),3.45
1,1100023,Ariquemes (RO),20.41
2,1100031,Cabixi (RO),4.80
3,1100049,Cacoal (RO),20.72
4,1100056,Cerejeiras (RO),6.12
...,...,...,...
5560,5222005,Vianópolis (GO),13.15
5561,5222054,Vicentinópolis (GO),10.00
5562,5222203,Vila Boa (GO),4.47
5563,5222302,Vila Propício (GO),2.36


In [4]:
population_df = pd.read_excel('data/IBGE/tabela6579.ods', engine='odf', skiprows=4, skipfooter=1, names=['city_id', 'city_and_state', 'population'])
population_df

,city_id,city_and_state,population
0,1100015,Alta Floresta D'Oeste (RO),22728
1,1100023,Ariquemes (RO),109523
2,1100031,Cabixi (RO),5188
3,1100049,Cacoal (RO),85893
4,1100056,Cerejeiras (RO),16204
...,...,...,...
5565,5222005,Vianópolis (GO),13977
5566,5222054,Vicentinópolis (GO),8873
5567,5222203,Vila Boa (GO),6312
5568,5222302,Vila Propício (GO),5882


In [5]:
vaccination_df = spark.read.csv('data/vaccines/*/*.csv', header=True, sep=';')

In [6]:
vaccination_df.limit(5).toPandas()

,document_id,paciente_id,paciente_idade,paciente_dataNascimento,paciente_enumSexoBiologico,paciente_racaCor_codigo,paciente_racaCor_valor,paciente_endereco_coIbgeMunicipio,paciente_endereco_coPais,paciente_endereco_nmMunicipio,...,vacina_lote,vacina_fabricante_nome,vacina_fabricante_referencia,vacina_dataAplicacao,vacina_descricao_dose,vacina_codigo,vacina_nome,sistema_origem,data_importacao_rnds,id_sistema_origem
0,4ebbccf0-89d2-40d4-aee5-41fea43e5823-i0b0,9d3c1e07bcec8d966cdbd6ac1ff7c7705c341a153a756e...,58,1962-11-22,F,03,PARDA,130260,10,MANAUS,...,210012,Butantan/Sinovac,None,2021-03-06T00:00:00.000Z,2ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,SMV,2021-03-10 00:48:00,18183
1,f99cb084-d62c-4e36-a837-072e69709de5-i0b0,6bfdd3d0b2ebc6025f85a43879722e5e7db56111cb79c2...,60,1960-07-12,F,99,SEM INFORMACAO,130260,10,MANAUS,...,210044,Butantan/Sinovac,None,2021-03-19T00:00:00.000Z,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,SMV,2021-03-22 00:25:37,18183
2,758116f5-ef60-48b3-81c4-17160239720d-i0b0,464c9a4e474a718ba01ee38f6c2555bed7bd08efd5255b...,68,1953-03-02,F,03,PARDA,130260,10,MANAUS,...,4120Z005,University of Oxford/AstraZeneca,None,2021-03-02T00:00:00.000Z,1ª Dose,85,Vacina Covid-19 - Covishield,SMV,2021-03-05 01:14:05,18183
3,64efc03f-553e-400d-8965-79fe6c03a263-i0b0,f1f47acda0184f09aee1500a84a848e71a0779211c1456...,41,1979-04-25,F,03,PARDA,130260,10,MANAUS,...,210038,Butantan/Sinovac,None,2021-03-22T00:00:00.000Z,2ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,SMV,2021-03-24 02:09:31,18183
4,e898d8d2-90b3-47ff-a445-afa9f3eb3f5e-i0b0,6175f1bd989a6e95995e1c8240cf258203a4623cff7880...,62,1958-09-30,M,99,SEM INFORMACAO,130260,10,MANAUS,...,210012,Butantan/Sinovac,None,2021-03-16T00:00:00.000Z,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,SMV,2021-03-18 00:44:55,18183


### Step 2: Explore and Assess the Data
#### Explore the Data 
We should the datasets to see if we have missing values and also explore some numerical fields to see if they make sense.

#### Cleaning Steps
If necessary, we'll perform some data cleaning after exploring.

### Demographic density dataset

In [7]:
print(demographic_density_df['city_id'].hasnans)
print(demographic_density_df['city_and_state'].hasnans)
print(demographic_density_df['demographic_density'].hasnans)

False
False
False


No null values found!

In [8]:
demographic_density_df['demographic_density'].describe()

count     5565.000000
mean       108.202122
std        572.444281
min          0.130000
25%         11.560000
50%         24.370000
75%         51.670000
max      13024.560000
Name: demographic_density, dtype: float64

In [9]:
max_density_idx = demographic_density_df['demographic_density'].idxmax()
demographic_density_df.iloc[max_density_idx]

city_id                                3305109
city_and_state         São João de Meriti (RJ)
demographic_density                   13024.56
Name: 3248, dtype: object

In [10]:
min_density_idx = demographic_density_df['demographic_density'].idxmin()
demographic_density_df.iloc[min_density_idx]

city_id                    1302108
city_and_state         Japurá (AM)
demographic_density           0.13
Name: 105, dtype: object

Given what we know about Brazil, it's safe to say that the data is OK! Using Wikipedia as reference, the values check!

Sao Joao do Meriti (RJ): https://pt.wikipedia.org/wiki/S%C3%A3o_Jo%C3%A3o_de_Meriti

Japura (AM): https://pt.wikipedia.org/wiki/Japur%C3%A1_(Amazonas)

### Population dataset

In [11]:
print(population_df['city_id'].hasnans)
print(population_df['city_and_state'].hasnans)
print(population_df['population'].hasnans)

False
False
False


No null values found!

In [12]:
population_df['population'].describe().apply(lambda x: format(x, '.2f'))

count        5570.00
mean        38017.18
std        222892.99
min           776.00
25%          5442.25
50%         11665.50
75%         25663.75
max      12325232.00
Name: population, dtype: object

In [13]:
max_population_idx = population_df['population'].idxmax()
population_df.iloc[max_population_idx]

city_id                  3550308
city_and_state    São Paulo (SP)
population              12325232
Name: 3829, dtype: object

In [14]:
min_population_idx = population_df['population'].idxmin()
population_df.iloc[min_population_idx]

city_id                         3166600
city_and_state    Serra da Saudade (MG)
population                          776
Name: 3026, dtype: object

The population data also check! Also leaving some references here:

Sao Paulo (SP): https://pt.wikipedia.org/wiki/S%C3%A3o_Paulo

Serra da Saudade (MG): https://pt.wikipedia.org/wiki/Serra_da_Saudade

### Vaccination dataset

Here we will look at just as few columns which are interesting to us: age (paciente_idade), IBGE city code (estabelecimento_municipio_codigo) and application date (vacina_dataAplicacao).

In [15]:
age_as_pandas_df = vaccination_df.select('paciente_idade').toPandas()
age_as_pandas_df.astype(int).describe()

,paciente_idade
count,1.180128e+06
mean,5.753505e+01
std,1.895849e+01
min,0.000000e+00
25%,4.100000e+01
50%,6.200000e+01
75%,7.200000e+01
max,1.260000e+02


Brazil started its vaccination campaign with healthcare workers and elders, so we expect high averages. The floor should be above 18 years old though, because the vaccines aren't approved yet for minors, therefore we will remove these rows when generating the final tables.

In [16]:
ibge_code_as_pandas_df = vaccination_df.select('estabelecimento_municipio_codigo').toPandas()
ibge_code_as_pandas_df['estabelecimento_municipio_codigo'].hasnans

False

We want evaluate vaccination rates from regions to cities, so it's important we have the IBGE code available.

In [17]:
vaccination_df.agg({'vacina_dataAplicacao': 'max'}).collect()

[Row(max(vacina_dataAplicacao)='2021-04-18T03:00:00.000Z')]

In [18]:
vaccination_df.agg({'vacina_dataAplicacao': 'min'}).collect()

[Row(min(vacina_dataAplicacao)='2021-01-01T00:00:00.000Z')]

The vaccination dates seem fine! Let's now convert the type of some columns.

In [19]:
vaccination_df.printSchema()

root
 |-- document_id: string (nullable = true)
 |-- paciente_id: string (nullable = true)
 |-- paciente_idade: string (nullable = true)
 |-- paciente_dataNascimento: string (nullable = true)
 |-- paciente_enumSexoBiologico: string (nullable = true)
 |-- paciente_racaCor_codigo: string (nullable = true)
 |-- paciente_racaCor_valor: string (nullable = true)
 |-- paciente_endereco_coIbgeMunicipio: string (nullable = true)
 |-- paciente_endereco_coPais: string (nullable = true)
 |-- paciente_endereco_nmMunicipio: string (nullable = true)
 |-- paciente_endereco_nmPais: string (nullable = true)
 |-- paciente_endereco_uf: string (nullable = true)
 |-- paciente_endereco_cep: string (nullable = true)
 |-- paciente_nacionalidade_enumNacionalidade: string (nullable = true)
 |-- estabelecimento_valor: string (nullable = true)
 |-- estabelecimento_razaoSocial: string (nullable = true)
 |-- estalecimento_noFantasia: string (nullable = true)
 |-- estabelecimento_municipio_codigo: string (nullable = 

In [20]:
vaccination_df = vaccination_df.withColumn('date', to_date('vacina_dataAplicacao')) \
                        .withColumn('birthdate', to_date('paciente_dataNascimento')) \
                        .withColumn('age', col('paciente_idade').cast(IntegerType())) \
                        .filter(vaccination_df.paciente_idade >= 18)

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model

Let's define the dimensions first. With the datasets available, there are four dimensions we can define:
- **Cities**: this table is a join of both IBGE tables, with columns for the IBGE code, demographic density and population.
- **Persons**: this table has data of all citizens that were vaccinated so far. It's important to notice that the dataset is already anonymized, so we can use them.
- **Dates**: this is a simple table with application dates.
- **Vaccines**: general data from each vaccine (which laboratory made it, first or second dose and so on)

And our fact table:
- **Vaccination**: referencing cities, persons, dates and vaccines. Other than foreign keys, this table will also have specific data about the application, such as which dose was applied.

#### 3.2 Fields
Let's now define the fields selected for each table:
- Cities: PK(city_id), name, population, demographic_density
- Persons: PK(person_id), age, birthdate, sex, race_color, residence_city_id
- Dates: PK(date), day, month, year, weekday
- Vaccines: PK(vaccine_id), manufacturer_id, manufacturer_name, vaccine_name
- Vaccination: PK(vaccination_id), city_id, person_id, date, vaccine_id, vaccine_batch, vaccine_dose, vaccine_group_name, vaccine_category_name

#### 3.3 Mapping Out Data Pipelines
To transform the data into our model, we need to load each raw dataset, perform transformations and finally load the dimensions and fact tables. For simplicity, we'll use the spark data lake project as reference, loading data from a bucket, transforming the table and storing parquet files also on a bucket. This will allow to execute locally in Python and on the cloud using AWS EMR, for instance.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

### Cities

In [21]:
# Converting dataframes from pandas to spark
demographic_density_df = spark.createDataFrame(demographic_density_df)
population_df = spark.createDataFrame(population_df)

In [22]:
cities_table = population_df.select(['city_id', 'population']).join(demographic_density_df, ['city_id'])

In [23]:
cities_table = cities_table.withColumn('city_name', split('city_and_state', ' \(').getItem(0)) \
                    .withColumn('state', split('city_and_state', '[\(\)]').getItem(1)) \
                    .withColumn('city_id', substring('city_id', 0, 6)) \
                    .drop('city_and_state')

Let's check a few rows of the table after the transformations!

Obs.: IBGE's code has seventh digit, for verification purposes. The vaccination data has only 6 digits on city codes, so we removed the last digit here.

In [24]:
cities_table.show(10)

+-------+----------+-------------------+--------------------+-----+
|city_id|population|demographic_density|           city_name|state|
+-------+----------+-------------------+--------------------+-----+
| 130115|     30846|               9.09|   Careiro da Várzea|   AM|
| 150345|     65625|               9.84|     Ipixuna do Pará|   PA|
| 170200|      8467|                1.7|            Araguaçu|   TO|
| 210637|     16511|              14.46|       Maranhãozinho|   MA|
| 211085|     11941|              13.76|São Francisco do ...|   MA|
| 220117|      3952|              14.63|   Barra D'Alcântara|   PI|
| 220520|     19170|              20.85|              Jaicós|   PI|
| 220910|      6247|               9.85| Santa Cruz do Piauí|   PI|
| 230690|     34636|              18.33|           Jaguaribe|   CE|
| 240020|     58384|              40.84|                 Açu|   RN|
+-------+----------+-------------------+--------------------+-----+
only showing top 10 rows



In [25]:
cities_table.write.partitionBy('state').parquet('output_data/cities', mode='overwrite')

### Persons

In [26]:
person_fields = [
    'paciente_id as person_id', 'age', 'birthdate',
    'paciente_enumSexoBiologico as sex', 'paciente_racaCor_valor as race_color',
    'paciente_endereco_coIbgeMunicipio as residence_city_id'
]

persons_table = vaccination_df.selectExpr(person_fields).dropDuplicates(['person_id'])

In [27]:
persons_table.show(10)

+--------------------+---+----------+---+--------------+-----------------+
|           person_id|age| birthdate|sex|    race_color|residence_city_id|
+--------------------+---+----------+---+--------------+-----------------+
|0000814f48f9d98cd...| 80|1940-08-14|  M|       AMARELA|           160023|
|00202ddb37a6a346a...| 67|1953-12-03|  F|SEM INFORMACAO|           530010|
|0020602d26e34ebca...| 45|1975-04-04|  F|       AMARELA|           120010|
|0024c4ff55f7538d1...| 39|1981-05-19|  F|SEM INFORMACAO|           130260|
|003d32596d4f043f0...| 33|1987-07-10|  F|       AMARELA|           160030|
|0042f9e801ee3b886...| 58|1962-11-19|  F|         PARDA|           130260|
|0049af6207fbfae24...| 25|1995-03-05|  F|         PARDA|           530010|
|004a45443bbe755c9...| 60|1960-05-09|  F|SEM INFORMACAO|           110020|
|005bafa860af3b402...| 71|1949-05-17|  M|SEM INFORMACAO|           530010|
|00756e9e114ab0efc...| 86|1934-02-11|  F|         PARDA|           130070|
+--------------------+---

In [28]:
persons_table.write.parquet('output_data/persons', mode='overwrite')

### Dates

In [29]:
date_fields = [
    'date', 'day(date) as day', 'month(date) as month', 'year(date) as year',
    'weekofyear(date) as week', 'dayofweek(date) as weekday'
]

dates_table = vaccination_df.dropDuplicates(['date']).selectExpr(date_fields)

In [30]:
dates_table.show(10)

+----------+---+-----+----+----+-------+
|      date|day|month|year|week|weekday|
+----------+---+-----+----+----+-------+
|2021-01-27| 27|    1|2021|   4|      4|
|2021-02-15| 15|    2|2021|   7|      2|
|2021-03-22| 22|    3|2021|  12|      2|
|2021-01-25| 25|    1|2021|   4|      2|
|2021-01-18| 18|    1|2021|   3|      2|
|2021-02-02|  2|    2|2021|   5|      3|
|2021-03-07|  7|    3|2021|   9|      1|
|2021-02-26| 26|    2|2021|   8|      6|
|2021-04-07|  7|    4|2021|  14|      4|
|2021-04-15| 15|    4|2021|  15|      5|
+----------+---+-----+----+----+-------+
only showing top 10 rows



In [31]:
dates_table.write.partitionBy(['year', 'month']).parquet('output_data/dates', mode='overwrite')

### Vaccines

In [32]:
vaccine_fields = [
    'vacina_codigo as vaccine_id', 'vacina_nome as vaccine_name',
    'vacina_fabricante_referencia as manufacturer_id', 'vacina_fabricante_nome as manufacturer_name'
]

vaccines_table = vaccination_df.selectExpr(vaccine_fields).dropDuplicates(['vaccine_id'])

In [33]:
vaccines_table.show()

+----------+--------------------+--------------------+--------------------+
|vaccine_id|        vaccine_name|     manufacturer_id|   manufacturer_name|
+----------+--------------------+--------------------+--------------------+
|        87|Vacina covid-19 -...|                null|BioNTech/Fosun Ph...|
|        85|Vacina Covid-19 -...|                null|University of Oxf...|
|        89|Covid-19-AstraZeneca|Organization/0039...| MINISTERIO DA SAUDE|
|        86|Covid-19-Coronava...|                null|    Butantan/Sinovac|
|        88|Vacina covid-19 -...|                null|       Janssen-Cilag|
+----------+--------------------+--------------------+--------------------+



In [34]:
vaccines_table.write.parquet('output_data/vaccines', mode='overwrite')

### Vaccination

In [35]:
vaccination_df.printSchema()

root
 |-- document_id: string (nullable = true)
 |-- paciente_id: string (nullable = true)
 |-- paciente_idade: string (nullable = true)
 |-- paciente_dataNascimento: string (nullable = true)
 |-- paciente_enumSexoBiologico: string (nullable = true)
 |-- paciente_racaCor_codigo: string (nullable = true)
 |-- paciente_racaCor_valor: string (nullable = true)
 |-- paciente_endereco_coIbgeMunicipio: string (nullable = true)
 |-- paciente_endereco_coPais: string (nullable = true)
 |-- paciente_endereco_nmMunicipio: string (nullable = true)
 |-- paciente_endereco_nmPais: string (nullable = true)
 |-- paciente_endereco_uf: string (nullable = true)
 |-- paciente_endereco_cep: string (nullable = true)
 |-- paciente_nacionalidade_enumNacionalidade: string (nullable = true)
 |-- estabelecimento_valor: string (nullable = true)
 |-- estabelecimento_razaoSocial: string (nullable = true)
 |-- estalecimento_noFantasia: string (nullable = true)
 |-- estabelecimento_municipio_codigo: string (nullable = 

In [36]:
vaccination_fields = [
    'estabelecimento_municipio_codigo as city_id', 'paciente_id as person_id',
    'date', 'vacina_codigo as vaccine_id', 'vacina_lote as vaccine_batch',
    'vacina_descricao_dose as vaccine_dose', 'vacina_grupoAtendimento_nome as vaccine_group_name',
    'vacina_categoria_nome as vaccine_category_name'
]

vaccination_table = vaccination_df.selectExpr(vaccination_fields) \
                    .withColumn('vaccination_id', monotonically_increasing_id())

In [37]:
vaccination_table.show(10)

+-------+--------------------+----------+----------+-------------+------------+--------------------+---------------------+--------------+
|city_id|           person_id|      date|vaccine_id|vaccine_batch|vaccine_dose|  vaccine_group_name|vaccine_category_name|vaccination_id|
+-------+--------------------+----------+----------+-------------+------------+--------------------+---------------------+--------------+
| 130260|9d3c1e07bcec8d966...|2021-03-06|        86|       210012|     2ª Dose|              Outros| Trabalhadores de ...|             0|
| 130260|6bfdd3d0b2ebc6025...|2021-03-19|        86|       210044|     1ª Dose|Pessoas de 60 a 6...|         Faixa Etária|             1|
| 130260|464c9a4e474a718ba...|2021-03-02|        85|     4120Z005|     1ª Dose|Pessoas de 65 a 6...|         Faixa Etária|             2|
| 130260|f1f47acda0184f09a...|2021-03-22|        86|       210038|     2ª Dose|  Pessoal da Limpeza| Trabalhadores de ...|             3|
| 130260|6175f1bd989a6e959...|2021

In [38]:
vaccination_table.write.parquet('output_data/vaccination', mode='overwrite')

#### 4.2 Data Quality Checks
Let's define the quality checks:
 * Integrity constraints for keys: primary keys on dimensions and the foreign keys on our fact table.
 * Count checks to ensure all tables have data.

In [39]:
cities_null_id = cities_table.where(cities_table.city_id.isNull()).collect()
assert len(cities_null_id) == 0

In [40]:
dates_null_id = dates_table.where(dates_table.date.isNull()).collect()
assert len(dates_null_id) == 0

In [41]:
persons_null_id = persons_table.where(persons_table.person_id.isNull()).collect()
assert len(persons_null_id) == 0

In [42]:
vaccines_null_id = vaccines_table.where(vaccines_table.vaccine_id.isNull()).collect()
assert len(vaccines_null_id) == 0

In [43]:
vaccination_null_city = vaccination_table.where(vaccination_table.city_id.isNull()).collect()
assert len(vaccination_null_city) == 0

In [44]:
vaccination_null_date = vaccination_table.where(vaccination_table.date.isNull()).collect()
assert len(vaccination_null_date) == 0

In [45]:
vaccination_null_person = vaccination_table.where(vaccination_table.person_id.isNull()).collect()
assert len(vaccination_null_person) == 0

In [46]:
vaccination_null_vaccine = vaccination_table.where(vaccination_table.vaccine_id.isNull()).collect()
assert len(vaccination_null_vaccine) == 0

In [47]:
assert cities_table.count() > 0
assert dates_table.count() > 0
assert persons_table.count() > 0
assert vaccines_table.count() > 0
assert vaccination_table.count() > 0

All quality checks passed!

#### 4.3 Data dictionary 

- **Cities**
    - city_id: IBGE's (Brazilian Institute of Geography and Statistics) unique id for each city in Brazil. The first two digits refer to the state and the following identify the city.
    - name: city name.
    - population: the city's population estimate.
    - demographic_density: the city's demographic density (population/total area).
- **Persons**
    - person_id: an unique id for each vaccinated person. This field is anonimized on the source.
    - age: person's age.
    - birthdate: date the person was born.
    - sex: the person's biological sex.
    - race_color: person's race/color description.
    - residence_city_id: IBGE's code of the city the person resides.
- **Dates**
    - date: a data, formatted as YYYY-MM-DD.
    - day: day of the month.
    - month.
    - year.
    - week: week of the year.
    - weekday: day of the week.
- **Vaccines**
    - vaccine_id: a numeric identifier of each vaccine available on the source.
    - vaccine_name: name given by the manufacturer.
    - manufacturer_id: unique id of each vaccine manufacturer.
    - manufacturer_name.
- **Vaccination**
    - vaccination_id: unique number for each dose applied.
    - vaccine_id: id of the applied vaccine.
    - city_id: IBGE's code of the city where the vaccine was applied.
    - person_id: vaccinated person id.
    - date: vaccination date.
    - vaccine_id: id of the specific vaccine applied.
    - vaccine_batch: the manufacturing batch of the applied vaccine.
    - vaccine_dose: which dose was applied (first or second).
    - vaccine_group_name: group to which the vaccinated person belongs.
    - vaccine_category_name: nationwide category of groups vaccinated.

#### Step 5: Complete Project Write Up
* Choice of tools and technologies for the project:
    - I chose spark because it's nearly as easy to use as pandas while handling large datasets better. The data lake architecture, similar to the one used on Project 4, makes it easy to develop the ETL and if deployed over a cluster, will be very efficient in processing the datasets.

* How often the data should be updated and why:
    - The vaccination data is updated on the source on a daily basis, so that would be the best period to update. This data could be used to support government officials decision making, to prioritize distribution of new vaccines and doses, so it's very important to keep the data updated as often as possible, to provide insights and demonstrate results of all actions taken.

* How I would approach the problem under the following scenarios:
 * The data was increased by 100x: I'd probably use a more robust storage solution, like Redshift, to make the data available quicker. While spark is great at processing, using a cloud data lake will probably decrease my analytics performance.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day: I would use a tool to orchestrate the pipeline, like Airflow, to make sure the processing is scheduled and I have control and monitoring tools to observe the status of the ETL, in case anything goes wrong. Being warned that the pipeline failed before the dashboard reports that is very important.
 * The database needed to be accessed by 100+ people: I would use a database like Redshift, because it's made for analytical queries and AWS has all the access management tools necessary to make sure the data is secure and accessed only by those who need to. Another solution I think is cool would be providing an API to allow users to access the data, if possible.